In [2]:
import pandas as pd
import argparse
import util
from helpers import query_dscf, query_intake, clean_sample_id
import regex as re

In [3]:
plog = pd.read_excel(util.print_log, sheet_name='LOG', header=0)


/Users/brianmonahan/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [4]:
plog.head()

,Date Printed,Study,PBMCs,Box numbers,Unnamed: 4,BOXES,KITS,Printed by,Screw cap tubes brand and batch #,4.5ml tubes brand and batch #,...,Notes,Number of boxes printed,Number of kits printed,EDTA,CPT,SST,ACD (yellow),Glass SST,Aliquot Tubes,PBMC Tubes
0,NaN,NaN,NaN,From,To,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-21 00:00:00,PARIS (CPT),No,1216,1221,Yes,Yes,"LS, KR",NaN,NaN,...,NaN,6.0,18.0,0.0,54.0,36.0,0.0,0.0,360.0,0.0
2,2021-05-21 00:00:00,MARS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-05-21 00:00:00,ATLAS,NaN,151,154,NaN,NaN,WM,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-05-21 00:00:00,PARIS (CPT),NaN,NaN,NaN,NaN,NaN,WM,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
plog.columns

Index(['Date Printed', 'Study', 'PBMCs', 'Box numbers', 'Unnamed: 4', 'BOXES',
       'KITS', 'Printed by', 'Screw cap tubes brand and batch #',
       '4.5ml tubes brand and batch #', 'Cryotubes brand and batch #',
       'tiger top sst brand, batch #', 'CPT/EDTA brand, batch #',
       '15ml conical brand and batch #', '50 ml conical brand and batch #',
       'red glass sst brand and batch #', 'ACD brand and batch #',
       'Sodium Heparin brand and batch #', 'FEEDBACK', 'Unnamed: 19', 'Notes',
       'Number of boxes printed', 'Number of kits printed', 'EDTA', 'CPT',
       'SST', 'ACD (yellow)', 'Glass SST', 'Aliquot Tubes', 'PBMC Tubes'],
      dtype='object')

In [69]:
plog['temp'] = plog['CPT/EDTA brand, batch #'].dropna(axis=0).apply(lambda x: re.sub(r"[# ]","",str(x))).apply(lambda x: re.sub("Vacutainer","",str(x)))
plog['temp'] = plog['temp'].dropna(axis=0).apply(lambda x: re.sub("lastbag-openednewone-","",str(x))).apply(lambda x: re.sub("openednewCPTstartingforC3206,","",str(x)))
plog['temp'] = plog['temp'].dropna(axis=0).apply(lambda x: re.sub("D,3","D3",str(x))).apply(lambda x: re.sub(",",";",str(x)))
test = plog['temp'].unique()
print(test)

[nan 'BD2110530;4/30/23' '2023-04-3000:00:00' '2173882' 'bd366643;2112268'
 'bd366643;2112268;2066315' 'bd366643;2066315' 'lot2230572;2023-08-31'
 'bd366643;lot2230572' 'bd366643;2189042' 'bd362753;2110526'
 'bd366643;2025400' 'bd362753;2110526/2173882' 'bd362753;2173882'
 'bd362753;lot2110530' 'bd366643;2084341' 'lot2084341'
 'bd362753;2110526/2054823' 'bd366643;1245677' 'bd362753;2110530'
 'bd366643;2189041' 'ref352753;lot2069215' 'bd362753;2336128'
 'bd362753;2069215' 'bd362753;2189041' 'bd362753;2110530&3142370'
 'bd362753;2189042' 'bd362753;3142370' 'BD362753;3230543'
 'BD362753;2173882' 'BD362753;2189042' 'BD362753;2110530'
 'BD362753;3263463' 'BD366643;3254840' 'BD362753;3263470'
 'BD366643;2189042' 'BD366643;LOT:3320807' 'BD366643;LOT:3254840'
 'Catalog:BD366643;Lot:3320807;Exp:20241130'
 'Catalog:BD362753;Lot:4113200;Exp:20250430'
 'Catalog:BD366643;Lot:3320807;Exp:2024-11-30'
 'Catalog:BD362753;Lot:4113200;Exp:2025-04-30'
 'Catalog:BD362753;Lot:4078972;Exp:2025-03-31'
 'ref36